In [1]:
!pip install langchain==0.2.0 --quiet
!pip install langchain-openai==0.1.7 --quiet
!pip install langchain-community==0.2.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Linking Multiple Chains Sequentially in LCEL

Here we will see how we can link several LLM Chains sequentially using LCEL.

Typically the output from one chain might go as input into the next chain and so on.

The overall chain would run each chain sequentially in order till we get the final output which can be a combination of intermediate outputs and inputs from the previous chains.

In [4]:
it_support_queue = [
    "I can't access my email. It keeps showing an error message. Please help.",
    "Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?",
    "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
    "我无法访问公司的网站。每次都显示错误信息。请帮忙解决。"
]

it_support_queue

["I can't access my email. It keeps showing an error message. Please help.",
 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。']

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Chain 1: Detect customer message language
prompt1 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Output the language of the message in one word only, e.g. Spanish

  Customer Message:
  ```{orig_msg}```
"""
prompt_template1 = ChatPromptTemplate.from_template(prompt1)
llm_chain1 = (prompt_template1
                  |
              chatgpt
                  |
              StrOutputParser())

In [6]:
prompt_template1

ChatPromptTemplate(input_variables=['orig_msg'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['orig_msg'], template='\n  Act as a customer support agent.\n  For the customer support message delimited below by triple backticks,\n  Output the language of the message in one word only, e.g. Spanish\n\n  Customer Message:\n  ```{orig_msg}```\n'))])

In [7]:
it_support_queue[1]

'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'

In [8]:
llm_chain1.invoke({'orig_msg': it_support_queue[1]})

'Spanish'

In [9]:
from langchain.schema.runnable import RunnablePassthrough

RunnablePassthrough.assign(orig_lang=llm_chain1).invoke({'orig_msg': it_support_queue[1]})

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish'}

In [10]:
# Chain 2: Translate Customer Message to English
prompt2 = """
  Act as a customer support agent.
  For the customer message and customer message language delimited below by triple backticks,
  Translate the customer message from the customer message language to English
  if customer message language is not in English,
  else return back the original customer message.

  Customer Message:
  ```{orig_msg}```
  Customer Message Language:
  ```{orig_lang}```
"""
prompt_template2 = ChatPromptTemplate.from_template(prompt2)
llm_chain2 = (prompt_template2
                  |
              chatgpt
                  |
              StrOutputParser())

In [13]:
RunnablePassthrough.assign(trans_msg=llm_chain2).invoke({'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish'})

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish',
 'trans_msg': "I'm having issues with the VPN. I can't connect to the company's network. Can you help me, please?"}

In [11]:
# Chain 3: Generate a resolution response in English
prompt3 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Generate an appropriate resolution response in English.

  Customer Message:
  ```{trans_msg}```
"""
prompt_template3 = ChatPromptTemplate.from_template(prompt3)
llm_chain3 = (prompt_template3
                  |
              chatgpt
                  |
              StrOutputParser())

In [12]:
# Chain 4: Translate resolution response from English to Customer's original language
prompt4 = """
  Act as a customer support agent.
  For the customer resolution response and target language delimited below by triple backticks,
  Translate the customer resolution response message from English to the target language
  if target language is not in English,
  else return back the original customer resolution response.

  Customer Resolution Response:
  ```{trans_response}```
  Target Language:
  ```{orig_lang}```
"""
prompt_template4 = ChatPromptTemplate.from_template(prompt4)
llm_chain4 = (prompt_template4
                  |
              chatgpt
                  |
              StrOutputParser())

In [14]:
from langchain.schema.runnable import RunnablePassthrough

final_chain = (
    RunnablePassthrough.assign(orig_lang=llm_chain1)
      |
    RunnablePassthrough.assign(trans_msg=llm_chain2)
      |
    RunnablePassthrough.assign(trans_response=llm_chain3)
      |
    RunnablePassthrough.assign(orig_response=llm_chain4)
)

In [15]:
{'orig_msg': it_support_queue[1]}

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'}

In [16]:
response = final_chain.invoke({'orig_msg': it_support_queue[1]})
response

{'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 'orig_lang': 'Spanish',
 'trans_msg': "I'm having issues with the VPN. I can't connect to the company's network. Can you help me, please?",
 'trans_response': "Resolution:\nI'm sorry to hear that you're having trouble with the VPN connection. To assist you further, could you please provide more details about the issue you're facing? Have you tried restarting the VPN client or your device? If not, please give that a try and let me know if it resolves the problem. If the issue persists, please reach out to our technical support team for further assistance. Thank you for your patience.",
 'orig_response': '```Resolución:\nLamento escuchar que estás teniendo problemas con la conexión VPN. Para ayudarte mejor, ¿podrías proporcionar más detalles sobre el problema que estás enfrentando? ¿Has intentado reiniciar el cliente VPN o tu dispositivo? Si no lo has hecho, por favor int

In [17]:
it_support_queue

["I can't access my email. It keeps showing an error message. Please help.",
 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?',
 "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
 '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。']

In [18]:
it_support_queue_formatted = [{'orig_msg': msg} for msg in it_support_queue]
it_support_queue_formatted

[{'orig_msg': "I can't access my email. It keeps showing an error message. Please help."},
 {'orig_msg': 'Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?'},
 {'orig_msg': "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer."},
 {'orig_msg': '我无法访问公司的网站。每次都显示错误信息。请帮忙解决。'}]

In [19]:
responses = final_chain.map().invoke(it_support_queue_formatted)

In [20]:
import pandas as pd
pd.DataFrame(responses)

,orig_msg,orig_lang,trans_msg,trans_response,orig_response
0,I can't access my email. It keeps showing an e...,English,I can't access my email. It keeps showing an e...,Resolution Response:\nI'm sorry to hear that y...,Resolution Response:\nI'm sorry to hear that y...
1,Tengo problemas con la VPN. No puedo conectarm...,Spanish,I'm having issues with the VPN. I can't connec...,Resolution Response:\nI'm sorry to hear that y...,Respuesta de resolución:\nLamento escuchar que...
2,Mon imprimante ne répond pas et n'imprime plus...,French,My printer is not responding and not printing ...,Resolution:\nI'm sorry to hear that you're exp...,Résolution:\nJe suis désolé d'apprendre que vo...
3,我无法访问公司的网站。每次都显示错误信息。请帮忙解决。,Chinese,I cannot access the company's website. It show...,Resolution:\nI apologize for the inconvenience...,```Resolution:\n对于您正在经历的不便，我感到抱歉。为了更好地帮助您，您能否提...
